In [1]:
# Clean the 'Data analyst Data.xlsx' All Events Data sheet using a robust pipeline and preview results
import pandas as pd
import numpy as np

In [2]:
# 1) Load
file_path_main = 'Data analyst Data.xlsx'
xl = pd.ExcelFile(file_path_main)


In [6]:
# 2) Standardize column names
new_cols = []
for c in df_raw.columns:
    c2 = str(c).strip().lower()
    c2 = c2.replace(' ', '_')
    c2 = c2.replace('-', '_')
    c2 = c2.replace('"', '')
    c2 = c2.replace("'", '')
    new_cols.append(c2)
df = df_raw.copy()
df.columns = new_cols

In [7]:
# Rename long question columns to concise names if present
rename_map = {
    'email_id': 'email',
    'attendee_status': 'status',
    'how_did_you_come_to_know_about_this_event?': 'how_did_you_know',
    'specify_in_others_(how_did_you_come_to_know_about_this_event)': 'others_specify',
    'year_of_graduation': 'grad_year',
    'experience_with_python_(months)': 'python_exp_months',
    'expected_salary_(lac)': 'expected_salary_lac',
    'leadership__skills': 'leadership_skills'
}
for k in list(rename_map.keys()):
    if k not in df.columns:
        # try alternate keys that may differ by punctuation
        for col in df.columns:
            if k.replace('?', '') == col.replace('?', ''):
                rename_map[col] = rename_map[k]
        if k in rename_map:
            rename_map.pop(k, None)

df = df.rename(columns=rename_map)

In [8]:
# 3) Trim strings
str_cols = df.select_dtypes(include=['object']).columns.tolist()
for c in str_cols:
    df[c] = df[c].astype(str).str.strip()
    df[c] = df[c].replace({'nan': np.nan, 'None': np.nan, '': np.nan})

In [9]:
# Lowercase identifiers for consistency
for c in ['email', 'events']:
    if c in df.columns:
        df[c] = df[c].str.lower()


In [10]:
# Title-case city
if 'city' in df.columns:
    df['city'] = df['city'].str.title()

In [11]:
# 4) Consolidate knowledge source
if 'how_did_you_know' in df.columns and 'others_specify' in df.columns:
    df['knowledge_source'] = df['how_did_you_know']
    mask_others = df['knowledge_source'].str.contains('others', case=False, na=False)
    df.loc[mask_others & df['others_specify'].notna(), 'knowledge_source'] = df.loc[mask_others, 'others_specify']


In [12]:
# 5) Family Income Mapping
if 'family_income' in df.columns:
    # Map family income to standardized scale
    income_mapping = {
        '0-2 Lakh': 1,
        '2-5 Lakh': 3.5,
        '5-7 Lakh': 6,
        '7+ Lakh': 8,
        }

    # Apply mapping for text values
    df['family_income'] = df['family_income'].replace(income_mapping)

    # Convert to numeric, handling any remaining text
    df['family_income'] = pd.to_numeric(df['family_income'], errors='coerce')

In [13]:
# 6) Expected Salary Cleaning
if 'expected_salary_lac' in df.columns:
    # Remove 'Lakhs' text and convert to numeric
    df['expected_salary_lac'] = df['expected_salary_lac'].astype(str).str.replace('Lakhs', '', case=False)
    df['expected_salary_lac'] = df['expected_salary_lac'].str.replace('Lac', '', case=False)
    df['expected_salary_lac'] = df['expected_salary_lac'].str.replace('L', '', case=False)
    df['expected_salary_lac'] = df['expected_salary_lac'].str.strip()

    # Convert to numeric
    df['expected_salary_lac'] = pd.to_numeric(df['expected_salary_lac'], errors='coerce')

In [15]:
 #7) Type conversions
numeric_specs = {
    'quantity': None,
    'grad_year': None,
    'cgpa': (0, 10),
    'python_exp_months': (0, None),
    'expected_salary_lac': (0, None)
}
for col, bounds in numeric_specs.items():
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        if bounds is not None:
            low, high = bounds
            if low is not None:
                df[col] = df[col].clip(lower=low)
            if high is not None:
                df[col] = df[col].clip(upper=high)

# Leadership skills to boolean
if 'leadership_skills' in df.columns:
    df['leadership_skills'] = df['leadership_skills'].str.lower().map({'yes': True, 'no': False})


In [16]:
# 8) Normalize status/events casing
for c in ['status', 'events']:
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip()

# 9) Deduplicate exact rows
before_dupes = len(df)
df = df.drop_duplicates()
after_dupes = len(df)
print('Dropped exact duplicate rows: ' + str(before_dupes - after_dupes))


Dropped exact duplicate rows: 0


In [17]:
# 10) Resolve conflicts by Email + Event
key_cols = [c for c in ['email', 'events'] if c in df.columns]

def resolve_col(series):
    name = series.name
    if pd.api.types.is_numeric_dtype(series):
        if name in ['cgpa']:
            return series.median()
        # discrete numeric fields - use mode then median fallback
        mode = series.mode(dropna=True)
        if not mode.empty:
            return mode.iloc[0]
        return series.median()
    # categorical
    mode = series.mode(dropna=True)
    if not mode.empty:
        return mode.iloc[0]
    return series.dropna().iloc[0] if series.dropna().shape[0] > 0 else np.nan

if len(key_cols) == 2:
    df_agg = df.groupby(key_cols, dropna=False).agg(resolve_col).reset_index()
else:
    df_agg = df.copy()

In [18]:
# 11) Plausibility filters

if 'python_exp_months' in df_agg.columns:
    df_agg.loc[df_agg['python_exp_months'] < 0, 'python_exp_months'] = np.nan

# 12) Quick summary and preview
print('Cleaned rows: ' + str(len(df_agg)))
print(df_agg.head(15))

# Save cleaned file
out_file = 'all_events_cleaned.csv'
df_agg.to_csv(out_file, index=False)
print('Saved cleaned dataset to file: ' + out_file)

Cleaned rows: 3900
                 email                                       events  \
0      10pawan@xyz.com            data visualization using power bi   
1      aaditya@xyz.com      internship program(ip) success conclave   
2       aaftab@xyz.com      internship program(ip) success conclave   
3    aakanksha@xyz.com                       art of resume building   
4    aakanksha@xyz.com            data visualization using power bi   
5    aakanksha@xyz.com  talk on skill and employability enhancement   
6     aakansha@xyz.com            data visualization using power bi   
7      aakassh@xyz.com                    the agile ways of working   
8   aaliyaruba@xyz.com                              hello ml and dl   
9        aamir@xyz.com  talk on skill and employability enhancement   
10       aarav@xyz.com            data visualization using power bi   
11       aarav@xyz.com  talk on skill and employability enhancement   
12       aarav@xyz.com             the sdlc & their transf